In [1]:
import rasterio

# Path to your clipped LULC file
lulc_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"
with rasterio.open(lulc_path) as src:
    bounds = src.bounds
    crs = src.crs
    width = src.width
    height = src.height
    res = src.res

print("CRS:", crs)
print("Bounding Box:")
print(f"  Min Lon: {bounds.left}")
print(f"  Min Lat: {bounds.bottom}")
print(f"  Max Lon: {bounds.right}")
print(f"  Max Lat: {bounds.top}")
print("Resolution (Lon, Lat):", res)
print(f"Pixel Dimensions: {width} x {height}")


CRS: EPSG:32644
Bounding Box:
  Min Lon: 108365.73527523666
  Min Lat: 3152714.2869458776
  Max Lon: 499835.73527523666
  Max Lat: 3491954.2869458776
Resolution (Lon, Lat): (30.0, 30.0)
Pixel Dimensions: 13049 x 11308


In [2]:
import rasterio
from rasterio.windows import from_bounds

# Path to your LULC file
lulc_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"

# Geographic bounds for N31_E80 tile
# Covers 31°N to 32°N latitude and 80°E to 81°E longitude
tile_min_lon, tile_min_lat = 80.0, 31.0
tile_max_lon, tile_max_lat = 81.0, 32.0

with rasterio.open(lulc_path) as src:
    # Reproject bounds to LULC CRS if needed
    if src.crs.to_string() != "EPSG:4326":
        from pyproj import Transformer
        transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
        tile_min_lon, tile_min_lat = transformer.transform(tile_min_lon, tile_min_lat)
        tile_max_lon, tile_max_lat = transformer.transform(tile_max_lon, tile_max_lat)

    # Clip window for the tile area
    try:
        window = from_bounds(tile_min_lon, tile_min_lat, tile_max_lon, tile_max_lat, src.transform)
    except ValueError:
        print("Tile is completely outside LULC raster.")
        exit()

    # Read data for that window
    data = src.read(1, window=window, masked=True)

    # Check if there are any valid pixels (not nodata)
    if data.mask.all():
        print("✅ No LULC coverage in N31_E80 — safe to skip DEM tile.")
    else:
        print("⚠ LULC coverage found in N31_E80 — DEM tile needed.")


⚠ LULC coverage found in N31_E80 — DEM tile needed.


In [2]:
import rasterio
from rasterio.transform import from_origin
import numpy as np

# Input .hgt file
hgt_file = r"C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif"

# Known lat/lon for this tile
lat = 31
lon = 80

# SRTM tiles are 3601x3601 with 1 arc-second resolution (~30m)
size = 3601

# Read binary elevation data
with open(hgt_file, "rb") as f:
    elevations = np.fromfile(f, np.dtype('>i2'), size * size).reshape((size, size))

# Transform for GeoTIFF
transform = from_origin(lon, lat + 1, 1/3600, 1/3600)  # 1 arc-second resolution

# Save as GeoTIFF
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=elevations.shape[0],
    width=elevations.shape[1],
    count=1,
    dtype=elevations.dtype,
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(elevations, 1)

print(f"✅ Converted {hgt_file} to {output_tif}")

TypeError: invalid dtype: dtype('>i2')

In [3]:
import rasterio
from rasterio.transform import from_origin
import numpy as np

# Input .hgt file
hgt_file = r"C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt"
output_tif = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif"


# Known lat/lon for this tile
lat = 31
lon = 80

# SRTM tiles are 3601x3601 with 1 arc-second resolution (~30m)
size = 3601

# Read binary elevation data (big-endian) and convert to native endianness
with open(hgt_file, "rb") as f:
    elevations = np.fromfile(f, np.dtype('>i2'), size * size).reshape((size, size)).astype(np.int16)

# Transform for GeoTIFF
transform = from_origin(lon, lat + 1, 1/3600, 1/3600)  # 1 arc-second resolution

# Save as GeoTIFF
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=elevations.shape[0],
    width=elevations.shape[1],
    count=1,
    dtype='int16',  # native dtype string
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(elevations, 1)

print(f"✅ Converted {hgt_file} to {output_tif}")


✅ Converted C:\Users\Ankit\Downloads\N31E081.SRTMGL1.hgt\N31E081.hgt to C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\DEM_datasets\N31E080.tif
